In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## مرحله ۱: تعریف مدل‌های Pydantic برای خروجی‌های ساختاریافته

این مدل‌ها **طرح**ی را تعریف می‌کنند که عوامل بازمی‌گردانند. استفاده از `response_format` با Pydantic تضمین می‌کند:
- ✅ استخراج داده ایمن از نظر نوع
- ✅ اعتبارسنجی خودکار
- ✅ بدون خطاهای تجزیه از پاسخ‌های متن آزاد
- ✅ مسیریابی آسان شرطی بر اساس فیلدها


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## مرحله ۲: ایجاد ابزار رزرو هتل

این ابزار همان چیزی است که **availability_agent** برای بررسی موجود بودن اتاق‌ها فراخوانی می‌کند. ما از تزئین‌کننده `@ai_function` استفاده می‌کنیم تا:
- یک تابع پایتون را به ابزاری قابل فراخوانی توسط هوش مصنوعی تبدیل کنیم
- به‌طور خودکار JSON schema برای LLM تولید کنیم
- اعتبارسنجی پارامترها را مدیریت کنیم
- امکان فراخوانی خودکار توسط عوامل را فراهم کنیم

برای این دمو:
- **استکهلم، سیاتل، توکیو، لندن، آمستردام** → اتاق موجود است ✅
- **تمام شهرهای دیگر** → اتاق موجود نیست ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## مرحله ۳: تعریف توابع شرطی برای مسیریابی

این توابع پاسخ عامل را بررسی کرده و تعیین می‌کنند که کدام مسیر در جریان کاری دنبال شود.

**الگوی کلیدی:**
1. بررسی کنید که آیا پیام `AgentExecutorResponse` است
2. خروجی ساختاریافته (مدل Pydantic) را تجزیه کنید
3. مقدار `True` یا `False` را برای کنترل مسیریابی بازگردانید

جریان کاری این شرایط را روی **لبه‌ها** ارزیابی می‌کند تا تصمیم بگیرد کدام اجراکننده را در مرحله بعد فراخوانی کند.


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## مرحله ۴: ایجاد اجراکننده نمایش سفارشی

اجراکننده‌ها اجزای جریان کاری هستند که تبدیل‌ها یا اثرات جانبی را انجام می‌دهند. ما از تزئین‌کننده `@executor` برای ایجاد یک اجراکننده سفارشی که نتیجه نهایی را نمایش می‌دهد استفاده می‌کنیم.

**مفاهیم کلیدی:**
- `@executor(id="...")` - یک تابع را به عنوان اجراکننده جریان کاری ثبت می‌کند
- `WorkflowContext[Never, str]` - اشاره‌های نوع برای ورودی/خروجی
- `ctx.yield_output(...)` - نتیجه نهایی جریان کاری را ارائه می‌دهد


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## مرحله ۵: بارگذاری متغیرهای محیطی

پیکربندی کلاینت LLM. این مثال با موارد زیر کار می‌کند:
- **مدل‌های GitHub** (سطح رایگان با توکن GitHub)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## مرحله ۶: ایجاد عوامل هوش مصنوعی با خروجی‌های ساختاریافته

ما **سه عامل تخصصی** ایجاد می‌کنیم که هر کدام در یک `AgentExecutor` قرار داده شده‌اند:

1. **availability_agent** - بررسی موجودی هتل با استفاده از ابزار
2. **alternative_agent** - پیشنهاد شهرهای جایگزین (در صورت نبود اتاق)
3. **booking_agent** - تشویق به رزرو (در صورت موجود بودن اتاق‌ها)

**ویژگی‌های کلیدی:**
- `tools=[hotel_booking]` - ابزار را در اختیار عامل قرار می‌دهد
- `response_format=PydanticModel` - خروجی JSON ساختاریافته را الزام می‌کند
- `AgentExecutor(..., id="...")` - عامل را برای استفاده در جریان کار بسته‌بندی می‌کند


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## مرحله ۷: ساخت جریان کاری با مسیرهای شرطی

اکنون از `WorkflowBuilder` برای ساخت نمودار با مسیرهای شرطی استفاده می‌کنیم:

**ساختار جریان کاری:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**روش‌های کلیدی:**
- `.set_start_executor(...)` - نقطه ورود را تنظیم می‌کند
- `.add_edge(from, to, condition=...)` - مسیر شرطی اضافه می‌کند
- `.build()` - جریان کاری را نهایی می‌کند


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## مرحله ۸: اجرای مورد آزمایشی ۱ - شهر بدون موجودی (پاریس)

بیایید مسیر **بدون موجودی** را با درخواست هتل‌ها در پاریس (که در شبیه‌سازی ما اتاقی ندارد) آزمایش کنیم.


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## مرحله ۹: اجرای مورد آزمایشی ۲ - شهر با موجودی (استکهلم)

حالا مسیر **موجودی** را با درخواست هتل‌ها در استکهلم (که در شبیه‌سازی ما اتاق دارد) آزمایش کنیم.


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## نکات کلیدی و مراحل بعدی

### ✅ آنچه آموخته‌اید:

1. **الگوی WorkflowBuilder**
   - از `.set_start_executor()` برای تعریف نقطه شروع استفاده کنید
   - از `.add_edge(from, to, condition=...)` برای مسیریابی شرطی استفاده کنید
   - با فراخوانی `.build()` جریان کاری را نهایی کنید

2. **مسیریابی شرطی**
   - توابع شرطی `AgentExecutorResponse` را بررسی می‌کنند
   - خروجی‌های ساختاریافته را برای تصمیم‌گیری در مسیریابی تجزیه کنید
   - با بازگرداندن `True` یک مسیر فعال می‌شود و با `False` از آن عبور می‌شود

3. **ادغام ابزارها**
   - از `@ai_function` برای تبدیل توابع پایتون به ابزارهای هوش مصنوعی استفاده کنید
   - عوامل به طور خودکار ابزارها را در صورت نیاز فراخوانی می‌کنند
   - ابزارها JSON تولید می‌کنند که عوامل می‌توانند آن را تجزیه کنند

4. **خروجی‌های ساختاریافته**
   - از مدل‌های Pydantic برای استخراج داده‌های نوع‌دار استفاده کنید
   - هنگام ایجاد عوامل، `response_format=MyModel` را تنظیم کنید
   - پاسخ‌ها را با `Model.model_validate_json()` تجزیه کنید

5. **اجراکننده‌های سفارشی**
   - از `@executor(id="...")` برای ایجاد اجزای جریان کاری استفاده کنید
   - اجراکننده‌ها می‌توانند داده‌ها را تغییر دهند یا اثرات جانبی ایجاد کنند
   - از `ctx.yield_output()` برای تولید نتایج جریان کاری استفاده کنید

### 🚀 کاربردهای واقعی:

- **رزرو سفر**: بررسی موجودی، پیشنهاد جایگزین‌ها، مقایسه گزینه‌ها
- **خدمات مشتری**: مسیریابی بر اساس نوع مشکل، احساسات، اولویت
- **تجارت الکترونیک**: بررسی موجودی، پیشنهاد جایگزین‌ها، پردازش سفارش‌ها
- **مدیریت محتوا**: مسیریابی بر اساس امتیازهای سمی، گزارش‌های کاربران
- **جریان‌های تأیید**: مسیریابی بر اساس مقدار، نقش کاربر، سطح ریسک
- **پردازش چندمرحله‌ای**: مسیریابی بر اساس کیفیت داده‌ها، کامل بودن

### 📚 مراحل بعدی:

- افزودن شرایط پیچیده‌تر (معیارهای چندگانه)
- پیاده‌سازی حلقه‌ها با مدیریت وضعیت جریان کاری
- افزودن زیرجریان‌ها برای اجزای قابل استفاده مجدد
- ادغام با API‌های واقعی (رزرو هتل، سیستم‌های موجودی)
- افزودن مدیریت خطا و مسیرهای جایگزین
- تصویری کردن جریان‌های کاری با ابزارهای تصویری داخلی



---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما تلاش می‌کنیم دقت را حفظ کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطاها یا نادرستی‌ها باشند. سند اصلی به زبان اصلی آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، توصیه می‌شود از ترجمه حرفه‌ای انسانی استفاده کنید. ما مسئولیتی در قبال سوء تفاهم‌ها یا تفسیرهای نادرست ناشی از استفاده از این ترجمه نداریم.
